In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

import numpy as np
import h5py

from micron2.data import stream_dataset
from matplotlib import pyplot as plt

input_size = 128
crop_size = 96
pth = '/home/ingn/tmp/micron2-data/pembroRT-set1-set2/merged_v2.hdf5'
# pth = '/storage/codex/preprocessed_data/210113_Breast_Cassette11_reg1/210113_Breast_Cassette11_reg1.hdf5'
with h5py.File(pth, 'r') as f:
    all_channels = [b.decode('UTF-8') for b in f['meta/channel_names'][:]]
    print(all_channels)
    
    # just use all the channels
    use_channels = all_channels

In [ ]:
def process(x):
    """
    x is [N, h, w, c]
    """
#     x = tf.cast(x, tf.float32)
#     x = (x - means) / stds
    
    x = tf.cast(x, tf.float32)#/255.
    x = tf.math.log1p(x)
    #x = tf.transpose(tf.image.per_image_standardization(tf.transpose(x)))
    return x

dataset = stream_dataset(pth, 
                         use_channels=use_channels,
                         group_name='images'
                        )
dataset = (dataset.map(process, num_parallel_calls=2)
           .batch(32)
           .prefetch(8)
          )

In [ ]:
def gen_data():
    for batch in dataset:
        batch = batch.numpy()
        yield batch
        
gen = gen_data()
batch = next(gen)
print(batch.shape)

In [ ]:
for i, ch in enumerate(use_channels):
    print(i, ch)

In [ ]:
ch = 22 
m = 1.75
r = 4
c = 8
fig, axs = plt.subplots(r,c, figsize=(c*m,r*m))

batch = next(gen)
print(np.max(batch), np.min(batch))
ch_max = np.max(batch[:,:,:,ch].ravel())
    
for i, ax in enumerate(axs.ravel()):
    img = batch[i,:,:,ch]
    ax.matshow(img, vmax=ch_max, vmin=0)
    ax.set_xticks([])
    ax.set_yticks([])